In [5]:


import numpy as np
import pandas as pd
import sklearn as sk
from sklearn.model_selection import train_test_split
data=pd.read_excel(r"C:\Users\Sijal\Desktop\New folder (2)\t2_w_filter_ucla.xlsx")
data = data.dropna()
X=data.drop(['UCLA Score (Similar to PIRADS v2)'], axis=1)
y=data['UCLA Score (Similar to PIRADS v2)']
# y=[]
# for i in Y:
#     if i<3:
#         y.append(0)
#     elif i==3:
#         y.append(1)
#     else:
#         y.append(2)





In [ ]:


import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.feature_selection import (
    SelectKBest, f_classif, VarianceThreshold, RFE, SelectFromModel
)
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import FactorAnalysis
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder
import warnings

def interpretable_modeling(X, y):
    # Suppress warnings
    warnings.filterwarnings("ignore")
    
    # Convert X and y to pandas DataFrame and Series
    if not isinstance(X, pd.DataFrame):
        X = pd.DataFrame(X)
    if not isinstance(y, pd.Series):
        y = pd.Series(y)
    
    # Handle missing values in X and y
    # Drop rows where y is NaN
    mask = y.notnull()
    X = X[mask].reset_index(drop=True)
    y = y[mask].reset_index(drop=True)
    
    # Identify categorical and numerical columns
    categorical_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
    numerical_cols = X.select_dtypes(include=['int64', 'float64', 'int32', 'float32']).columns.tolist()
    
    # Handle missing values and encode features
    # Impute numerical columns with mean
    numerical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler())
    ])

    # Impute categorical columns with most frequent and encode
    # Updated parameter to 'sparse_output' for scikit-learn >= 1.0
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
    ])

    # Combine transformers
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_transformer, numerical_cols),
            ('cat', categorical_transformer, categorical_cols)
        ]
    )

    # Encode target variable if it's categorical
    if y.dtype == 'object' or y.dtype.name == 'category':
        label_encoder = LabelEncoder()
        y = label_encoder.fit_transform(y)
    else:
        y = y.values  # Ensure y is a numpy array

    # Define classifiers and feature selectors
    classifiers = {
        'Decision Tree': DecisionTreeClassifier(random_state=123),
        'Logistic Regression': LogisticRegression(random_state=123, max_iter=1000),
        'K-Nearest Neighbors': KNeighborsClassifier(),
        'Gaussian Naive Bayes': GaussianNB(),
        'Linear Discriminant Analysis': LinearDiscriminantAnalysis()
    }

    feature_selectors = {
        'SelectKBest (ANOVA F-value)': SelectKBest(f_classif, k='all'),
        'Variance Threshold': VarianceThreshold(threshold=0.0),
        'L1-based feature selection': SelectFromModel(
            LogisticRegression(C=1.0, penalty='l1', solver='liblinear', random_state=123)
        ),
        'Recursive Feature Elimination': RFE(
            estimator=LogisticRegression(random_state=123), n_features_to_select=None
        )
    }

    # Interpretable feature reducer
    feature_reducer = FactorAnalysis()

    random_seed = 123
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_seed)

    # Create a full pipeline with preprocessing and modeling
    def full_pipeline(clf, fs):
        return Pipeline(steps=[
            ('preprocessor', preprocessor),
            ('feature_reducer', feature_reducer),
            ('feature_selector', fs),
            ('classifier', clf)
        ])

    results = []
    
    for clf_name, clf in classifiers.items():
        for fs_name, fs in feature_selectors.items():
            pipeline = full_pipeline(clf, fs)
            
            try:
                scores = cross_val_score(pipeline, X, y, cv=cv, error_score='raise')
                results.append({
                    'Classifier': clf_name,
                    'Feature Selector': fs_name,
                    'Accuracy': np.mean(scores),
                    'Std': np.std(scores)
                })
                print(f"{clf_name} with {fs_name}: Mean accuracy={np.mean(scores):.4f}, Std={np.std(scores):.4f}")
            except Exception as e:
                print(f"Error with {clf_name} and {fs_name}: {e}")

    # Convert results to a DataFrame for easier viewing
    results_df = pd.DataFrame(results)
    print("\nSummary of Results:")
    print(results_df)
    return results_df

# Call the function with your data
results_df = interpretable_modeling(X, y)









Decision Tree with SelectKBest (ANOVA F-value): Mean accuracy=0.3762, Std=0.0125
